# Proyecto Minería de Datos

**Objetivo General:** Crear un modelo con Machine Learning que pronostique tendencias, para relacionar productos que sean ideales para ventas cruzadas, en base a las transacciones de clientes.

**Dataset:** https://www.kaggle.com/mathchi/online-retail-ii-data-set-from-ml-repository

**Problema:** Para una empresa cualquiera, le es importante mejorar sus ventas o servicios de acuerdo a las necesidades del cliente. Los grandes negocios implementan procesos inteligentes acorde a las acciones de sus clientes, para lograr mejorar sus ingresos, algunos de estos procedimientos están basados en los conocimientos de marketing, Big Data, Smart Data, estadística. Sin embargo para lograr el objetivo de aumentar las ventas, es necesario tener una base de datos de las transacciones realizadas para el análisis de los datos, por tal motivo en este proyecto se enfoca en la creación de un modelo que pueda
predecir tendencias de agrupaciones de productos en base a las compras realizadas por diferentes clientes en una tienda de regalos del Reino Unido entre el 2009 y 2011.

**Tema:** Modelo de pronóstico de tendencias para ventas cruzadas de una tienda de regalos del
Reino Unido entre el 2009 a 2011

**Librerías**

In [1]:
# Usada para la importación de los datos
import pandas as pd
# Usada para la graficación
import matplotlib.pyplot as plt

## Análisis de los datos

En este apartado se hace un análisis de los datos, para saber si existen datos nulos, el número de registros, distribución de los datos.

**Importación del dataset**

In [5]:
#Función para la lectura del dataset
def importdata():
    df = pd.read_csv('dataset/Year_2009-2011.csv',encoding='unicode_escape')
    return df

In [21]:
#Función para ver si existen datos nulos
def datosnull(df):
    existen_bol=1
    total_register=len(df.index)
    total_nan_values = df.isnull().sum().sum()
    existen=total_register-total_nan_values
    if(existen != 0):
        print("Número de datos nulos: ", total_nan_values)
        print("\nNúmero de registros finales: ", existen)
        df=eraseregis(df)
        return existen_bol, df
    else:
        return 0

In [22]:
data=importdata()
existen_bol,df=datosnull(data)

Número de datos nulos:  239426

Número de registros finales:  793610


In [19]:
#Función para graficar con puntos
def grapscat(x,y,color):
    plt.scatter(x, y, alpha=0.3, c=color)
    plt.show

## Selección de las variables

En esta sección se escogen las variables x & y del dataset

In [19]:
x=df[['Customer ID','Quantity','StockCode']]
y=df['StockCode']

## Limpieza y transformación

En esta sección se eliminan los registros con datos nulos, a partir de la selección de variables.

**Eliminación de datos nulos**

In [20]:
#Función para eliminación de registros con datos nulos
def eraseregis(data):
    data = data.dropna()
    return data

In [23]:
x

,Customer ID,Quantity,StockCode
0,13085.0,12,85048
1,13085.0,12,79323P
2,13085.0,12,79323W
3,13085.0,48,22041
4,13085.0,24,21232
...,...,...,...
1033031,12680.0,6,22899
1033032,12680.0,4,23254
1033033,12680.0,4,23255
1033034,12680.0,3,22138


In [24]:
existen_bol,df=datosnull(x)

Número de datos nulos:  0

Número de registros finales:  797885
